In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('log2 3.csv')

# Display the first few rows
df.head()


,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
# Define the target and features
X = df.drop(columns=['Action_allow', 'Action_deny', 'Action_drop'])
y = df[['Action_allow', 'Action_deny', 'Action_drop']].values.argmax(axis=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [64]:
from sklearn.feature_selection import SelectKBest, chi2
import pandas as pd

# Apply Chi-square test
chi2_scores, p_values = chi2(X_train, y_train)

# Create a DataFrame to display the features with their Chi-square scores and p-values
chi2_results = pd.DataFrame({'Feature': X.columns, 'Chi2 Score': chi2_scores, 'p-value': p_values})
print("Chi-square Test Results:\n", chi2_results)


Chi-square Test Results:
                  Feature   Chi2 Score        p-value
0            Source Port   363.299611   1.289709e-79
1       Destination Port  4398.077991   0.000000e+00
2        NAT Source Port  5744.377676   0.000000e+00
3   NAT Destination Port   785.429068  2.794119e-171
4                  Bytes     5.440085   6.587195e-02
5             Bytes Sent     8.984359   1.119621e-02
6         Bytes Received     5.360908   6.853204e-02
7                Packets     5.915074   5.194670e-02
8     Elapsed Time (sec)   184.343237   9.340600e-41
9              pkts_sent     5.836415   5.403044e-02
10         pkts_received     5.955828   5.089890e-02


In [65]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Initialize the model
logreg = LogisticRegression(max_iter=10000, solver='lbfgs', multi_class='multinomial')

# Initialize RFE
rfe = RFE(logreg, n_features_to_select=1)  # Select all features to get rankings

# Fit RFE
rfe.fit(X_train, y_train)

# Create a DataFrame to display the features with their ranking
rfe_results = pd.DataFrame({'Feature': X.columns, 'Ranking': rfe.ranking_})
rfe_results_sorted = rfe_results.sort_values(by='Ranking')

print("RFE Results (Sorted by Ranking):\n", rfe_results_sorted)


RFE Results (Sorted by Ranking):
                  Feature  Ranking
2        NAT Source Port        1
1       Destination Port        2
3   NAT Destination Port        3
0            Source Port        4
8     Elapsed Time (sec)        5
5             Bytes Sent        6
10         pkts_received        7
7                Packets        8
4                  Bytes        9
6         Bytes Received       10
9              pkts_sent       11


In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SequentialFeatureSelector as SFS
import pandas as pd

# Initialize the model
logreg = LogisticRegression(max_iter=10000, solver='lbfgs', multi_class='multinomial')

# Forward Feature Selection
ffs = SFS(logreg, 
          n_features_to_select='auto',  # Select features automatically based on cross-validation score
          direction='forward', 
          scoring='accuracy',
          cv=5)

ffs.fit(X_train, y_train)

# Get the inclusion order
ffs_features = list(X_train.columns[ffs.get_support()])
ffs_inclusion_order = {feature: idx for idx, feature in enumerate(ffs_features)}

# Create a DataFrame to display the features with their inclusion order
ffs_results = pd.DataFrame({'Feature': list(ffs_inclusion_order.keys()), 'Inclusion Order': list(ffs_inclusion_order.values())})
print("FFS Results:\n", ffs_results)


FFS Results:
                 Feature  Inclusion Order
0           Source Port                0
1      Destination Port                1
2       NAT Source Port                2
3  NAT Destination Port                3
4    Elapsed Time (sec)                4


In [67]:
logreg = LogisticRegression(max_iter=10000, solver='lbfgs', multi_class='multinomial')

# Perform backward selection using SequentialFeatureSelector
sfs2 = SFS(logreg, n_features_to_select='auto', direction='backward', scoring='accuracy', cv=5)
sfs2.fit(X_train, y_train)
sfs2_support = sfs2.get_support()

# Convert the results to a DataFrame
df_results = pd.DataFrame({'feature_names': X.columns.tolist(), 'support': sfs2_support})
print(df_results)

# Get the selected features
sfs2_features = X.loc[:, sfs2_support].columns.tolist()
print(f"Optimal subset size using backward feature selection: {len(sfs2_features)}")
print("Selected features:", sfs2_features)


           feature_names  support
0            Source Port     True
1       Destination Port     True
2        NAT Source Port     True
3   NAT Destination Port     True
4                  Bytes    False
5             Bytes Sent    False
6         Bytes Received    False
7                Packets    False
8     Elapsed Time (sec)     True
9              pkts_sent    False
10         pkts_received     True
Optimal subset size using backward feature selection: 6
Selected features: ['Source Port', 'Destination Port', 'NAT Source Port', 'NAT Destination Port', 'Elapsed Time (sec)', 'pkts_received']


In [68]:
from sklearn.linear_model import LogisticRegressionCV
import numpy as np
import pandas as pd

# Apply Lasso for feature selection
lasso = LogisticRegressionCV(cv=5, penalty='l1', solver='saga', multi_class='multinomial', max_iter=10000)
lasso.fit(X_train, y_train)

# Get the coefficients
lasso_coef = lasso.coef_

# Sum the absolute values of the coefficients across classes (if multiclass)
lasso_coef_sum = np.abs(lasso_coef).sum(axis=0)

# Create a DataFrame to display the features with their coefficients
lasso_results = pd.DataFrame({'Feature': X.columns, 'Coefficient': lasso_coef_sum})
print("Lasso Results:\n", lasso_results)


Lasso Results:
                  Feature  Coefficient
0            Source Port    10.314124
1       Destination Port    26.299015
2        NAT Source Port    49.010796
3   NAT Destination Port    79.200080
4                  Bytes     0.000000
5             Bytes Sent     2.215650
6         Bytes Received     0.000000
7                Packets     0.000000
8     Elapsed Time (sec)  1117.874386
9              pkts_sent     0.000000
10         pkts_received     0.000000
